In [1]:
from sage.all import *
from sage.symbolic.operators import add_vararg, mul_vararg

$\overline{\cdot}$ denoted the involution.

In [2]:
bar = sage.symbolic.function_factory.function('bar', nargs=1, print_latex_func=lambda _, arg: '\\overline{{{}}}'.format(latex(arg)))

In [3]:
lmbd = var('lmbd', latex_name='\\lambda')
mu = var('mu', latex_name='\\mu')
a, b = var('a b')
y, d = var('y d')
eps = var('eps', latex_name='\\varepsilon')
variables = [lmbd, a, b, y, d, eps, mu]

The utility functions defined below allow to manipulate the expressions involving $\tau$, reduction of expressions mod 2 and mapping those over matrices.

In [4]:
# taken with changes from https://wiki.sagemath.org/symbolics/rewrite

def mapexpression(expr, fct, level, addDepth=0, mulDepth=0):
     def mapex(expr, depth):               # a very local function
        if expr.is_integer():
            return expr
        opor = expr.operator()
        opands = expr.operands()
        if (opor is None):
            return expr       # a leaf in the expression tree
        if (opor == operator.add):           # recursive call thru sum
            opands = map(lambda ex: mapex(ex, depth + addDepth), opands)
            return sum(opands)
        if (opor == operator.mul):           # recursive call thru mul
            opands = map(lambda ex: mapex(ex, depth + mulDepth), opands)
            return prod(opands)
        if (level == -1) or (level[-1] >= depth):  # recursive call over operands
            opands = map(lambda ex: mapex(ex, depth + 1), opands)
        if level == -1 or depth in level:  # root of the subtree must be changed
            return fct(opor, opands)
        return opor(*opands)  # opands may or may not be changed by a recursive call
     return mapex(expr, 0)


# a way to circumvent a conflict between built-in `pow` functions
sagepow = (1/x).operator()
    

def bar_involution_rules(opor, opands):
    """
    bar(a ± b) => bar(a) ± bar(b)
    bar(a * b^n) => bar(a) * bar(b)^n
    bar(p/q) => p/q
    bar(lambda) = -lambda
    bar(mu) = 1 - mu
    """
    opands = list(opands)
    if opor == bar:
        opand = opands[0]
        opand_opor = opand.operator()
        if opand_opor is not None:
            opand_opands = opand.operands()
            if opand_opor == add_vararg or opand_opor == mul_vararg:
                return opand_opor(*[bar(o) for o in opand_opands])
            elif opand_opor == sagepow:
                return pow(bar(opand_opands[0]), opand_opands[1])
            elif opand_opor == bar:
                return opand_opands[0]
        if opand.is_numeric():
            return opand
        if opand == lmbd:
            return -lmbd
        if opand == mu:
            return 1 - mu
    return opor(*opands)


def rewrite(expr):
    stable = false
    while not stable:
        new_expr = mapexpression(expr, bar_involution_rules, -1)
        stable = (new_expr == expr)
        expr = new_expr
    return expr


def map_rewrite(m):
    return m.apply_map(rewrite)


def map_simplify_rational(m):
    return m.apply_map(lambda e: e.simplify_rational(algorithm='simple'))


def reduce_mod2(expr):
    tempvars = list(var(['bar_{}'.format(v) for v in variables]))
    numer_denom = []
    for e in expr.numerator_denominator():
        e = e.subs({bar(v): bar_v for v, bar_v in zip(variables, tempvars)})
        Rmod2 = PolynomialRing(GF(2), variables + tempvars)
        e = Rmod2(e)
        e = e.change_ring(ZZ)
        e = e.subs(**{str(bar_v): bar(v) for bar_v, v in zip(tempvars, variables)})
        numer_denom.append(e)
    return numer_denom[0] / numer_denom[1]


def map_reduce_mod2(m):
    return m.apply_map(lambda e: reduce_mod2(e))

In [5]:
j = matrix([[0, 0, 1], [0, 1, 0], [1, 0, 0]])


def is_in_su3(g, char2=False):
    rel = map_simplify_rational(map_rewrite(g.T * j * g.apply_map(bar)))
    if char2:
        rel = map_reduce_mod2(rel)
    return rewrite(expand(g.det())) == 1 and rel == j

In [6]:
w = matrix([[0, 0, 1],[0, -1, 0], [1, 0, 0]])


def xp(a, b):
    mat = matrix([
        [1, a, b],
        [0, 1, -bar(a)],
        [0, 0, 1]
    ])
    return mat


def xm(a, b):
    return w * xp(a, b) * w


def h(eps):
    return diagonal_matrix([eps, bar(eps)/eps, 1/bar(eps)])


show(xp(a, b), xm(a, b), h(lmbd))

[      1       a       b]
[      0       1 -bar(a)]
[      0       0       1] [     1      0      0]
[bar(a)      1      0]
[     b     -a      1] [          lmbd              0              0]
[             0 bar(lmbd)/lmbd              0]
[             0              0    1/bar(lmbd)]

Checking that $x_+(a,b)$ lives in $S\cdot{}^g S$ for some $g$, separately in case $\operatorname{char}(F)\neq 2$ and $=2$.

In [7]:
g = xp(a, b)
s1 = h(2) * xp(0, -a*bar(a)/4-b/2)
s2 = h(1/2)

s1gs2ginv = s1 * g * s2 * g^(-1)
show(s1gs2ginv, map_rewrite(s1gs2ginv))

[                                                                    1                                                      4*a*bar(1/2) - a    4*a*bar(1/2)*bar(a) - a*bar(a) - b - 1/2*(a*bar(a) - 2*b)/bar(1/2)]
[                                                                    0                                                       bar(2)*bar(1/2) bar(2)*bar(1/2)*bar(a) - 1/2*(bar(2)*bar(0) + bar(2)*bar(a))/bar(1/2)]
[                                                                    0                                                                     0                                                   1/(bar(2)*bar(1/2))] [      1       a       b]
[      0       1 -bar(a)]
[      0       0       1]

In [8]:
g = xp(a * lmbd, d)
# s1 = matrix([[1 + 1/lmbd, 0, y], [0, 1, 0], [0, 0, lmbd/(1 + lmbd)]])
s1 = h((lmbd+1)/lmbd) * xp(0, y)
s2 = h(lmbd/(1 + lmbd))

values = {
    y: (d + b * lmbd**2 + a * bar(a) * lmbd**3) / (lmbd**2 + 1)
}

s1gs2ginv = map_reduce_mod2(map_simplify_rational(map_rewrite(s1 * g * s2 * g^(-1))))

s1gs2ginv_eval = map_reduce_mod2(map_simplify_rational(map_rewrite(s1gs2ginv.subs(values))))

show(s1gs2ginv, s1gs2ginv_eval)

[                                                                                                    1                                                                                                     a (a*lmbd^4*bar(a) + a*lmbd^3*bar(a) + lmbd^3*y + lmbd^2*y + d*lmbd + lmbd*y + d + y)/(lmbd^3 + lmbd^2)]
[                                                                                                    0                                                                                                     1                                                                     (lmbd*bar(a) + bar(a))/(lmbd + 1)]
[                                                                                                    0                                                                                                     0                                                                                                     1] [     1      a      b]
[     0      1 bar(a)]
[     0      0      1]

It remains to check that $s_1\in\operatorname{SU}_3$, namely, to check that $\overline{y}+y=0$.

In [9]:
y_obstacle = rewrite((values[y]+bar(values[y]))).simplify_rational()
show(y_obstacle)
y_obstacle = y_obstacle.subs({
    b+bar(b): a*bar(a),
    bar(d): (lmbd*a)*bar(lmbd*a) - d
})
show(rewrite(y_obstacle))

((b + bar(b))*lmbd^2 + d + bar(d))/(lmbd^2 + 1)

0

Checking that $H < S\cdot{}^g S$ for some $g$. Again, separately for $\operatorname{char}(F) \neq 2$ and $=2$.

In [10]:
eta = bar(eps) / eps

In [11]:
g = map_rewrite(xp(-1, -1/2) * w * xp(1, -1/2))
s1 = matrix([[(eps+bar(eps))/2, 0, (eps-bar(eps))/4], [0, 1, 0], [1/bar(eps)-1/eps, 0, (1/eps+1/bar(eps))/2]])

s2 = matrix([[(eta+bar(eta))/2, 0, (eta-bar(eta))/4], [0, 1, 0], [eta-bar(eta), 0, (eta+bar(eta))/2]])
s2 = map_rewrite(s2)

print(is_in_su3(s1), is_in_su3(s2))

show(g, s1, s2)

s1gs2ginv = map_simplify_rational(map_rewrite(s1 * g * s2 * g^(-1)))

show(s1gs2ginv)

True True


[-1/2  1/2  1/4]
[   1    0  1/2]
[   1    1 -1/2] [1/2*eps + 1/2*bar(eps)                      0 1/4*eps - 1/4*bar(eps)]
[                     0                      1                      0]
[   -1/eps + 1/bar(eps)                      0 1/2/eps + 1/2/bar(eps)] [ 1/2*eps/bar(eps) + 1/2*bar(eps)/eps                                    0 -1/4*eps/bar(eps) + 1/4*bar(eps)/eps]
[                                   0                                    1                                    0]
[        -eps/bar(eps) + bar(eps)/eps                                    0  1/2*eps/bar(eps) + 1/2*bar(eps)/eps]

[         eps            0            0]
[           0 bar(eps)/eps            0]
[           0            0   1/bar(eps)]

In [12]:
g = map_reduce_mod2(map_rewrite(xp(1, mu) * w * xp(1, mu)))
s1 = matrix([[(mu+1)*eps+mu*bar(eps), 0, mu*(mu+1)*(eps+bar(eps))], [0, 1, 0], [1/eps+1/bar(eps), 0, (mu+1)/eps+mu/bar(eps)]])
s2 = matrix([[mu*eta+(mu+1)*bar(eta), 0, (mu**2+mu)*(eta+bar(eta))], [0, 1, 0], [eta+bar(eta), 0, (mu+1)*eta+mu*bar(eta)]])

print(is_in_su3(s1, char2=True), is_in_su3(s2, char2=True))

show(g, s1, s2)

s1gs2ginv = map_reduce_mod2(map_simplify_rational(map_rewrite(s1 * g * s2 * g^(-1))))

show(s1gs2ginv)

True True


[    mu mu + 1   mu^2]
[     1      0 mu + 1]
[     1      1     mu] [  eps*(mu + 1) + mu*bar(eps)                            0 (eps + bar(eps))*(mu + 1)*mu]
[                           0                            1                            0]
[          1/eps + 1/bar(eps)                            0   (mu + 1)/eps + mu/bar(eps)] [  (mu + 1)*bar(bar(eps)/eps) + mu*bar(eps)/eps                                              0 (mu^2 + mu)*(bar(eps)/eps + bar(bar(eps)/eps))]
[                                             0                                              1                                              0]
[              bar(eps)/eps + bar(bar(eps)/eps)                                              0   mu*bar(bar(eps)/eps) + (mu + 1)*bar(eps)/eps]

[         eps            0            0]
[           0 bar(eps)/eps            0]
[           0            0   1/bar(eps)]